# Investigating upward propagating lee wave behavior


In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import ray_tracer as ray
import matplotlib.pyplot as plt
import numpy as np
from jupyterthemes import jtplot
import pandas as pd
from ipywidgets import FloatProgress
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual
import xarray as xr
import gsw
import seaborn as sns
import cmocean
# %connect_info # shows kernel id for connecting an ipython console 
from matplotlib.ticker import FormatStrFormatter
from scipy.interpolate import interp2d
from matplotlib import rcParams
plt.rcParams.update({'font.size':18})
rcParams['font.sans-serif'] = "Times New Roman"
rcParams['font.family'] = "sans-serif"
%connect_info # lets you open a connected ipython terminal (not necessary if you are using lab instead of notebook)


{
  "shell_port": 65358,
  "iopub_port": 65359,
  "stdin_port": 65360,
  "control_port": 65361,
  "hb_port": 65362,
  "ip": "127.0.0.1",
  "key": "013e5e03-fe4237ad7b2080727c887830",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-2a352d1d-f431-475d-b813-47831bb2e639.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [4]:
wave_properties = pd.read_pickle('CCW_dom_waves.pkl')
# load ray tracer and generate field functions using center field as mean of lat and long and depth
X = [wave_properties['lon'].mean(), wave_properties['lat'].mean(), 1000]
K = [wave_properties['k'].mean(), wave_properties['l'].mean(),
     wave_properties['m'].mean(), wave_properties['omega'].mean()]
t0 = wave_properties['date'].mean()

pad = 5

rt = ray.raytracer(X, K, t0, lonpad=pad, latpad=pad, tpad=21)
wave_properties

,k,l,m,omega,depth,lon,lat,date,p_prime,Rw,stn
0,0.000467,-0.000248,0.014842,-0.000228,2048.0,-49.474120,-53.761580,734558.0,0.021973,1.880952,1.0
1,0.000580,-0.000353,0.006249,-0.000359,1024.0,-49.465520,-53.758380,734558.0,0.015918,1.087331,2.0
2,0.000486,-0.000268,0.008202,-0.000322,2560.0,-49.236908,-53.562284,734559.0,0.028313,1.902460,6.0
3,0.000204,0.000032,0.008202,-0.000176,1024.0,-49.193775,-53.525682,734559.0,0.013254,0.935789,7.0
5,0.000344,0.000017,0.003125,-0.000415,512.0,-49.122354,-53.464639,734559.0,0.016821,0.900596,9.0
6,0.003658,0.001825,0.008202,-0.001458,3072.0,-49.080900,-53.421400,734559.0,0.035103,0.973536,10.0
7,0.001266,0.000195,0.008983,-0.000398,2560.0,-49.049306,-53.394546,734559.0,0.024673,1.618385,11.0
8,0.001966,-0.000515,0.025387,-0.000580,512.0,-48.999472,-53.332845,734559.0,0.011745,0.901281,13.0
9,0.000410,0.000360,0.010155,-0.000247,1536.0,-48.935919,-53.241273,734560.0,0.021963,0.935584,15.0
10,0.000287,-0.000087,0.006640,-0.000187,2048.0,-48.935919,-53.241273,734560.0,0.013667,1.302135,15.0


In [70]:
wave = wave_properties.loc[11]
wave.depth = 1024
wave.k = 0.00076
wave.l = -0.0003992948345679147
wave

sensitivity = 0.2
num = 5

# Run sensitivitry 

In [71]:
KK = np.linspace(wave.k*(1-sensitivity), wave.k*(1+sensitivity), num=num)
LL = np.linspace(wave.l*(1-sensitivity), wave.l*(1+sensitivity), num=num)
WW = np.linspace(wave.omega*(1-sensitivity), wave.omega*(1+sensitivity), num=num)
MM = np.linspace(wave.m*(1-sensitivity), wave.m*(1+sensitivity), num=num)


In [82]:
tstep = 600
# duration = 150
duration = 200
direction = 'forward'
runs = []
for K in KK:
    rt.K0 = [K, wave.l, -wave.m, wave.omega]
    rt.X0 = [wave.lon, wave.lat, wave.depth]
    rt.t0 = wave.date
    rt.p0 = wave.p_prime
    runs.append(rt.run(tstep=tstep,
                 duration=duration,
                 direction=direction,
                 strain=True,
                 stops=True,
                 clearance=.5,
                 vertspeed=False,
                 time_constant=False,
                 progress_bar=False))
Ksens = pd.concat(runs)

runs = []
for L in LL:
    rt.K0 = [wave.k, L, -wave.m, wave.omega]
    rt.X0 = [wave.lon, wave.lat, wave.depth]
    rt.t0 = wave.date
    rt.p0 = wave.p_prime
    runs.append(rt.run(tstep=tstep,
                 duration=duration,
                 direction=direction,
                 strain=True,
                 stops=True,
                 clearance=.5,
                 vertspeed=False,
                 time_constant=False,
                 progress_bar=False))
Lsens = pd.concat(runs)

runs = []
for M in MM:
    rt.K0 = [wave.k, wave.l, -M, wave.omega]
    rt.X0 = [wave.lon, wave.lat, wave.depth]
    rt.t0 = wave.date
    rt.p0 = wave.p_prime
    runs.append(rt.run(tstep=tstep,
                 duration=duration,
                 direction=direction,
                 strain=True,
                 stops=True,
                 clearance=.5,
                 vertspeed=False,
                 time_constant=False,
                 progress_bar=False))
Msens = pd.concat(runs)

In [83]:
runs = []
for W in WW:
    rt.K0 = [wave.k, wave.l, -wave.m, W]
    rt.X0 = [wave.lon, wave.lat, wave.depth]
    rt.t0 = wave.date
    rt.p0 = wave.p_prime
    runs.append(rt.run(tstep=tstep,
                 duration=duration,
                 direction=direction,
                 strain=True,
                 stops=True,
                 clearance=.5,
                 vertspeed=False,
                 time_constant=False,
                 progress_bar=False))
Wsens = pd.concat(runs)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(Ksens.Lon, Ksens.Lat, c='r', alpha=.1)
plt.scatter(Lsens.Lon, Lsens.Lat, c='g', alpha=.05)
plt.scatter(Msens.Lon, Msens.Lat, c='b', alpha=.01)
plt.scatter(Wsens.Lon, Wsens.Lat, c='m', alpha=.01)
plt.title('Overhead Path')

Kaspect = np.sqrt(Ksens.k**2 + Ksens.l**2 + Ksens.m**2)
Laspect = np.sqrt(Lsens.k**2 + Lsens.l**2 + Lsens.m**2)
Maspect = np.sqrt(Msens.k**2 + Msens.l**2 + Msens.m**2)
Waspect = np.sqrt(Wsens.k**2 + Wsens.l**2 + Wsens.m**2)

plt.figure(figsize=(10,10))
plt.scatter(Ksens.time, Kaspect, c='r', alpha=.1)
plt.scatter(Lsens.time, Laspect, c='g', alpha=.1)
plt.scatter(Msens.time, Maspect, c='b', alpha=.1)
plt.scatter(Wsens.time, Waspect, c='m', alpha=.1)
plt.title('Aspect')

plt.figure(figsize=(10,10))
plt.scatter(Ksens.time, Ksens.k, c='r', alpha=.1)
plt.scatter(Lsens.time, Lsens.k, c='g', alpha=.05)
plt.scatter(Msens.time, Msens.k, c='b', alpha=.01)
plt.scatter(Wsens.time, Wsens.k, c='m', alpha=.01)

plt.figure(figsize=(10,10))
plt.scatter(Ksens.time, Ksens.horiz_momentum, c='r', alpha=.1)
plt.scatter(Lsens.time, Lsens.horiz_momentum, c='g', alpha=.05)
plt.scatter(Msens.time, Msens.horiz_momentum, c='b', alpha=.01)
plt.scatter(Wsens.time, Wsens.horiz_momentum, c='m', alpha=.01)
plt.title('Horizontal Momentum Flux')

plt.figure(figsize=(10,10))
plt.scatter(Ksens.time, np.abs((2*np.pi)/Ksens.k), c='r', alpha=.1)
plt.scatter(Lsens.time, np.abs((2*np.pi)/Lsens.k), c='g', alpha=.05)
plt.scatter(Msens.time, np.abs((2*np.pi)/Msens.k), c='b', alpha=.01)
plt.scatter(Wsens.time, np.abs((2*np.pi)/Wsens.k), c='m', alpha=.01)
plt.title('Horizontal Wavelength (x)')

plt.figure(figsize=(10,10))
plt.scatter(Ksens.time, np.abs((2*np.pi)/Ksens.l), c='r', alpha=.1)
plt.scatter(Lsens.time, np.abs((2*np.pi)/Lsens.l), c='g', alpha=.05)
plt.scatter(Msens.time, np.abs((2*np.pi)/Msens.l), c='b', alpha=.01)
plt.scatter(Wsens.time, np.abs((2*np.pi)/Wsens.l), c='m', alpha=.01)
plt.title('Horizontal Wavelength (y)')

In [91]:
Msens.to_csv('m_sensitivity.csv')
Ksens.to_csv('k_sensitivity.csv')
Lsens.to_csv('l_sensitivity.csv')
Wsens.to_csv('omega_sensitivity.csv')